<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/U_net_lung_plus_cov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BASICS 
import os
import numpy as np
import torch

# Colab 
from pydrive.drive import GoogleDrive
from google.colab import drive
from google.colab import files
from pydrive.auth import GoogleAuth
from google.colab import auth
import torchvision.transforms as transforms
from oauth2client.client import GoogleCredentials
drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# MODEL and TRAIN
import torchvision
import torchvision.models
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

Mounted at /content/drive


In [ ]:
# The scripts contains some important functions have copied from the drive files in to the our script
!cp /content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/functions.ipynb  /content
!cp -R /content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/model /content

In [ ]:
# running  .ipynb scripts to call their functions
%run functions.ipynb
from model.unet import UNet, PretrainedUNet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The Calling the validaton dataset
lung_test=Lung_Dataset(split = "Test",class_folder='lung')# The length of the validation dataset to calculate the average value of the loss and accuracy
lung_test_dataset_size=lung_test.__len__()
# The Train dataset 
covid_test=Lung_Dataset(split = "Test",class_folder='covid')
covid_test_dataset_size=covid_test.__len__()
print(lung_test_dataset_size)
print(covid_test_dataset_size)
# Call cuda, if there is no GPU use CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# The model is called by this function from the model.py file 
net_lung = PretrainedUNet( in_channels=1, out_channels=2,batch_norm=True).to(device)
checkpoint = torch.load('/content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/trained_model_txts/model_lung_pre_70_epoch.pt')
net_lung.load_state_dict(checkpoint['model_state_dict'])



net_covid = PretrainedUNet( in_channels=1, out_channels=2,batch_norm=True).to(device)

checkpoint = torch.load('/content/drive/Shareddrives/FurkanDataandModels/Furkan_covid/codes/trained_model_txts/model_covid_pre_70_epoch.pt')
net_covid.load_state_dict(checkpoint['model_state_dict'])

6788
1166


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

<All keys matched successfully>

In [ ]:
batch_size=16
# loading images and masks for mini epochs 

covid_test_loader = torch.utils.data.DataLoader(
        covid_test, batch_size=batch_size,  num_workers=2, drop_last=False)

In [ ]:
# The allocaton for accuracy data variables
acc_covid_bin=0
acc_lung_bin=0


In [ ]:
for batch_idx, sample in enumerate(covid_test_loader):
    # Validation data preparing  and calling 
    # calcel gradient feature 
    net_lung.eval()
    net_covid.eval()
    # Call iamge and mask
    imgs , true_masks = sample['image'],sample['mask']

    imgs = imgs.to(device=device)
    true_masks=Negative_mask(true_masks,imgs.size(dim=0))
    true_masks = true_masks[:,0,:,:].to(device=device)

    with torch.set_grad_enabled(False):
        # Test Evaluation
        # output of the lung model
        masks_pred_lung = net_lung(imgs)[:,0,:,:]
        # output of the covid model
        masks_pred_covid = net_covid(imgs)[:,0,:,:]
        masks_pred_covid = masks_pred_covid.to(device=device)
        masks_pred_lung = masks_pred_lung.to(device=device)
        # acording to treshold create a mask with lung output
        masks_pred_lung= torch.where(masks_pred_lung<=0.5, 0, 1)
        # Use the mask on the covid output to eliminate the covid detection out of the lung
        masks_pred=masks_pred_covid*masks_pred_lung

 
        # Record the 
        acc_lung_bin +=binary_acc( masks_pred, true_masks)


print('Lung Acc bin :',(acc_lung_bin)/(covid_test_dataset_size/batch_size))



Lung Acc dice: inf
Lung Acc bin : tensor(0.9626, device='cuda:0')
Lung Acc iou : tensor(0.3554, device='cuda:0')
